# import


In [16]:
import sys
import warnings
warnings.filterwarnings('ignore')


import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import matplotlib.pyplot as plt
import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'
import seaborn as sns

root_path = f".."
data_path = f"{root_path}/data"
data_olist_path = f"{data_path}/olist"

In [17]:
def get_var_name(variable):
    globals_dict = globals()

    return [var_name for var_name in globals_dict if globals_dict[var_name] is variable]

class EDA:

    def __init__(
            self:pd.DataFrame,
            df,
            idx=None,
            x=None,
            y=None,
            fig_col_len=2,
            ) -> pd.DataFrame:

        self.idx = idx
        if idx != None:
            temp = df.set_index(idx)
            self.df = temp
        else:
            self.df = df

        if x == None:
            self.x = df.columns.to_list()
        else:
            self.x = x
            
        self.y = y
        self.fig_col_len = fig_col_len
        
    def print_title(body, br=2, bp="┌▣ ", hr=" ---- ---- ---- ----"):

        """
        body : 내용
        bp : bullet point, 글머리 기호
        hr : Horizontal Rule, 수평선
        """

        class ff:
            PURPLE = "\033[95m"
            CYAN = "\033[96m"
            DARKCYAN = "\033[36m"
            BLUE = "\033[94m"
            GREEN = "\033[92m"
            YELLOW = "\033[93m"
            RED = "\033[91m"
            BOLD = "\033[1m"
            UNDERLINE = "\033[4m"
            END = "\033[0m"

        print("\n" * br + ff.BOLD + bp + ff.UNDERLINE + body + ff.END + hr)
        

    def Check_df(df):

        EDA.print_title("""df.shape""")
        print(df.shape)


        EDA.print_title("""df.info()""")
        print(df.info())

        EDA.print_title("""df.head()""")
        display(df.head())

        EDA.print_title("""df.columns.to_list()""")
        print(df.columns.to_list())
        
    def Check(self) -> pd.DataFrame:

        df = self.df
        EDA.Check_df(df)    
        
    def Summary_df(df):
        
        EDA.print_title("""df.describe().T""")
        display(df.describe().T)

        try:
            EDA.print_title("""df.describe(include=['O'])""")
            display(df.describe(include=["O"]))
        except:
            pass

        # 결측치
        df_temp = df.isna()
        EDA.print_title("""df.isna().sum()""")
        display(df_temp.sum().to_frame())

        EDA.print_title("""sns.heatmap(data=df.isna())""")
        sns.heatmap(data=df.isna(), cmap='Greys')
        plt.title("""sns.heatmap(data=df.isna())""", fontsize=20)
        plt.show()

        EDA.print_title("df.isna().mean()")
        display(df_temp.mean().to_frame())

    def Summary(self):
        
        df = self.df
        EDA.Summary_df(df)

    def mv(self) -> None:

        df = self.df

        fig_col_len = self.fig_col_len
        import math
        fig_row_len = math.ceil(len(self.x) / fig_col_len)
        
        sns.pairplot(data=df, height=len(self.x)*4)
        

        df_corr = df.corr()
        EDA.print_title("""df.corr()""")
        display(df_corr)
        EDA.print_title("""sns.heatmap(data=df.corr(), annot=True, fmt=".2f")""")
        plt.figure(figsize=(20,7))
        sns.heatmap(data=df_corr, annot=True, fmt=".2f", annot_kws={"size": 14}, cmap="Blues")
        plt.show()
        

# Sort

## Data Load

In [18]:
df = pd.read_csv(f"{data_olist_path}/complete_orders.csv", index_col=0)

col_list = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
    "review_score",
]
dt_list = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
]
deri_list = ["delivered_purchase", "estimated_purchase", "delivered_estimated", "delivered_carrier", "carrier_purchase"]
dict_title = {
    deri_list[0]: "배송기간_실제",
    deri_list[1]: "배송기간_예상",
    deri_list[2]: "배송기간_실제예상차이",
    deri_list[3]: "배송원에게 전달된 후 소비자가 배송받은 시간",
    deri_list[4]: "구매 후 배송원에게 전달된 시간",
}


for i in dt_list:
    df[i] = pd.to_datetime(df[i])

df["delivered_purchase"] = (
    df["order_delivered_customer_date"] - df["order_purchase_timestamp"]
)

df["estimated_purchase"] = (
    df["order_estimated_delivery_date"] - df["order_purchase_timestamp"]
)

df["delivered_estimated"] = (
    df["order_delivered_customer_date"] - df["order_estimated_delivery_date"]
)

df["delivered_carrier"] = (
    df["order_delivered_customer_date"] - df["order_delivered_carrier_date"]
)

df["carrier_purchase"] = (
    df["order_delivered_carrier_date"] - df["order_purchase_timestamp"]
)

# EDA.Check_df(df)
# EDA.Summary_df(df)

EDA.Check_df(df)



┌▣ df.shape ---- ---- ---- ----
(92657, 33)


┌▣ df.info() ---- ---- ---- ----
<class 'pandas.core.frame.DataFrame'>
Index: 92657 entries, e481f51cbdc54678b7cc49136f2d6af7 to 66dea50a8b16d9b4dee7af250b4be1a5
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype          
---  ------                         --------------  -----          
 0   customer_id                    92657 non-null  object         
 1   order_status                   92657 non-null  object         
 2   order_purchase_timestamp       92657 non-null  datetime64[ns] 
 3   order_approved_at              92657 non-null  datetime64[ns] 
 4   order_delivered_carrier_date   92657 non-null  datetime64[ns] 
 5   order_delivered_customer_date  92657 non-null  datetime64[ns] 
 6   order_estimated_delivery_date  92657 non-null  datetime64[ns] 
 7   customer_unique_id             92657 non-null  object         
 8   zip_code_prefix_customer       92657 non-null  int64          
 9   cust

customer_id  \
order_id                                                             
e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
128e10d95713541c87cd1a2e48201934  a20e8105f23924cd00833fd87daa0831   
0e7e841ddf8f8f2de2bad69267ecfbcf  26c7ac168e1433912a51b924fbd34d34   
bfc39df4f36c3693ff3b63fcbea9e90a  53904ddbea91e1e92b2b3f1d09a7af86   
53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   

                                 order_status order_purchase_timestamp  \
order_id                                                                 
e481f51cbdc54678b7cc49136f2d6af7    delivered      2017-10-02 10:56:33   
128e10d95713541c87cd1a2e48201934    delivered      2017-08-15 18:29:31   
0e7e841ddf8f8f2de2bad69267ecfbcf    delivered      2017-08-02 18:24:47   
bfc39df4f36c3693ff3b63fcbea9e90a    delivered      2017-10-23 23:26:46   
53cdb2fc8bc7dce0b6741e2150273451    delivered      2018-07-24 20:41:37   

                                   order_approved_at  \
order_id                                               
e481f51cbdc54678b7cc49136f2d6af7 2017-10-02 11:07:15   
128e10d95713541c87cd1a2e48201934 2017-08-15 20:05:16   
0e7e841ddf8f8f2de2bad69267ecfbcf 2017-08-02 18:43:15   
bfc39df4f36c3693ff3b63fcbea9e90a 2017-10-25 02:14:11   
53cdb2fc8bc7dce0b6741e2150273451 2018-07-26 03:24:27   

                                 order_delivered_carrier_date  \
order_id                                                        
e481f51cbdc54678b7cc49136f2d6af7          2017-10-04 19:55:00   
128e10d95713541c87cd1a2e48201934          2017-08-17 15:28:33   
0e7e841ddf8f8f2de2bad69267ecfbcf          2017-08-04 17:35:43   
bfc39df4f36c3693ff3b63fcbea9e90a          2017-10-27 16:48:46   
53cdb2fc8bc7dce0b6741e2150273451          2018-07-26 14:31:00   

                                 order_delivered_customer_date  \
order_id                                                         
e481f51cbdc54678b7cc49136f2d6af7           2017-10-10 21:25:13   
128e10d95713541c87cd1a2e48201934           2017-08-18 14:44:43   
0e7e841ddf8f8f2de2bad69267ecfbcf           2017-08-07 18:30:01   
bfc39df4f36c3693ff3b63fcbea9e90a           2017-11-07 18:04:59   
53cdb2fc8bc7dce0b6741e2150273451           2018-08-07 15:27:45   

                                 order_estimated_delivery_date  \
order_id                                                         
e481f51cbdc54678b7cc49136f2d6af7                    2017-10-18   
128e10d95713541c87cd1a2e48201934                    2017-08-28   
0e7e841ddf8f8f2de2bad69267ecfbcf                    2017-08-15   
bfc39df4f36c3693ff3b63fcbea9e90a                    2017-11-13   
53cdb2fc8bc7dce0b6741e2150273451                    2018-08-13   

                                                customer_unique_id  \
order_id                                                             
e481f51cbdc54678b7cc49136f2d6af7  7c396fd4830fd04220f754e42b4e5bff   
128e10d95713541c87cd1a2e48201934  3a51803cc0d012c3b5dc8b7528cb05f7   
0e7e841ddf8f8f2de2bad69267ecfbcf  ef0996a1a279c26e7ecbd737be23d235   
bfc39df4f36c3693ff3b63fcbea9e90a  e781fdcc107d13d865fc7698711cc572   
53cdb2fc8bc7dce0b6741e2150273451  af07308b275d755c9edb36a90c618231   

                                  zip_code_prefix_customer  customer_city  \
order_id                                                                    
e481f51cbdc54678b7cc49136f2d6af7                      3149      sao paulo   
128e10d95713541c87cd1a2e48201934                      3366      sao paulo   
0e7e841ddf8f8f2de2bad69267ecfbcf                      2290      sao paulo   
bfc39df4f36c3693ff3b63fcbea9e90a                     88032  florianopolis   
53cdb2fc8bc7dce0b6741e2150273451                     47813      barreiras   

                                  ... freight_value  review_score  \
order_id                          ...                               
e481f51cbdc54678b7cc49136f2d6af7  ...         8.720             4   
128e10d95713541c87cd1a2e48201934



┌▣ df.columns.to_list() ---- ---- ---- ----
['customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'customer_unique_id', 'zip_code_prefix_customer', 'customer_city', 'customer_state', 'geolocation_lat_customer', 'geolocation_lng_customer', 'seller_id', 'zip_code_prefix_seller', 'seller_city', 'seller_state', 'geolocation_lat_seller', 'geolocation_lng_seller', 'order_item_id', 'product_id', 'shipping_limit_date', 'price', 'freight_value', 'review_score', 'payment_type', 'payment_installments', 'product_category_name_english', 'delivered_purchase', 'estimated_purchase', 'delivered_estimated', 'delivered_carrier', 'carrier_purchase']


In [24]:
df.groupby("customer_state")["delivered_purchase"].mean().sort_values(ascending=0).to_frame().head(30)

delivered_purchase
customer_state                           
RR             29 days 23:01:00.157894737
AP             27 days 02:55:05.060606060
AM             26 days 07:42:19.154929577
AL             24 days 05:59:26.277922078
PA             23 days 17:08:01.553097345
SE             21 days 12:54:25.948328267
MA             21 days 12:04:57.308588064
CE             21 days 07:24:18.868613138
AC             21 days 07:10:34.702702702
PB             20 days 11:23:08.315682281
PI             19 days 14:20:39.405286343
RO             19 days 13:09:29.944206008
RN             19 days 07:36:17.221006564
BA             19 days 06:45:37.195582586
PE             18 days 10:50:57.039062500
MT             18 days 01:00:59.965801886
TO             17 days 20:15:33.314393939
MS             15 days 15:28:55.301169590
ES             15 days 14:16:59.144723093
GO             15 days 13:51:35.369330453
RS             15 days 08:44:13.953000582
RJ             15 days 07:19:22.637460960
SC             14 days 22:44:51.008815750
DF             13 days 01:00:06.049891540
MG             12 days 01:18:11.393495190
PR             12 days 00:08:36.389053254
SP              8 days 19:10:12.265236095

# df_pe

In [19]:
df_pe = df[df["customer_state"] == "PE"]

In [20]:
df_pe.groupby("customer_city")["freight_value"].mean().sort_values(ascending=0).to_frame().head(10)

freight_value
customer_city                        
lagoa do carro                107.630
sanharo                       107.620
saloa                         101.673
trindade                       80.350
escada                         64.973
ouricuri                       60.484
iati                           59.417
belem do sao francisco         58.260
terra nova                     57.940
sao domingos                   55.070

In [21]:
df_pe.groupby("customer_city")["customer_unique_id"].count().sort_values(ascending=0).to_frame().head(10)

customer_unique_id
customer_city                              
recife                                  568
jaboatao dos guararapes                 128
olinda                                   88
petrolina                                59
caruaru                                  57
paulista                                 40
camaragibe                               32
garanhuns                                30
cabo de santo agostinho                  21
salgueiro                                16

In [31]:
df_pe.groupby("customer_city")["delivered_purchase"].mean().sort_values(ascending=0).to_frame().head(10)

delivered_purchase
customer_city                        
passira              50 days 00:55:50
jatauba              46 days 18:03:27
sao domingos         35 days 21:11:29
terezinha            35 days 17:28:11
cupira               32 days 23:59:43
pedra                31 days 13:48:48
serrita              31 days 12:01:32
pombos               31 days 07:34:30
amaraji              31 days 05:57:11
gameleira     30 days 19:12:29.250000

In [ ]:
df_pe.groupby("customer_city")["delivered_purchase"].mean().sort_values(ascending=0).to_frame().head(50)

In [ ]:
df_pe.groupby("customer_city")["delivered_purchase"].mean().sort_values(ascending=0).to_frame().head(50)